# 1. Levenshtein-distance 

In [189]:
def distance(w1, w2, i, j):
    if i == 0: return j
    if j == 0: return i

    # ef stafirnir eru eins þá þurfum við ekki að gera neitt 
    if w1[i - 1] == w2[j - 1]: return distance(w1, w2, i - 1, j - 1)
    
    possibilities = [
        distance(w1, w2, i, j - 1),
        distance(w1, w2, i - 1, j),
        distance(w1, w2, i - 1, j - 1)
    ]

    return min(possibilities) + 1

def dist(w1, w2):
    d = distance(w1, w2, len(w1), len(w2))
    print(f"fjarlægð á milli {w1} og {w2}: {d}")

In [190]:
dist("kisa", "peysa")
dist("ofurtumi", "tumi_tigur")
dist("írakabakarí", "ribbaldri")
dist("abcd", "dcba")

fjarlægð á milli kisa og peysa: 3
fjarlægð á milli ofurtumi og tumi_tigur: 9
fjarlægð á milli írakabakarí og ribbaldri: 7
fjarlægð á milli abcd og dcba: 4


# 2. Dictionary lookup

In [191]:
import re
# skiptum textanum okkar í einstök orð
with open("../data/log.txt", "r") as f:
    text = f.read()
    words = text.split()

f = re.compile("^\w+$")
words = [w for w in list(set(words)) if f.match(w)]

In [192]:
def find_errors(sent):
    errors = []
    for w in sent.split():
        if w not in words:
            errors.append(w)
    return errors

def show_errors(sent):
    errors = find_errors(sent)
    print(f"{len(errors)} villur í \"{sent}\"")
    print(f"villur: {errors}")
    print()

In [193]:
show_errors("Þeta er einn stórur villa")
show_errors("ég er bara ótrúlega góur að skrifa")
show_errors("Get ég skrifað setningu án villna")
show_errors("svo virðist ekki vera")

2 villur í "Þeta er einn stórur villa"
villur: ['Þeta', 'stórur']

1 villur í "ég er bara ótrúlega góur að skrifa"
villur: ['góur']

2 villur í "Get ég skrifað setningu án villna"
villur: ['setningu', 'villna']

0 villur í "svo virðist ekki vera"
villur: []



# 3. ICE BERT ICE BERT ICE BERT

In [194]:
from transformers import pipeline
icebert = pipeline("fill-mask", model="mideind/IceBERT")

In [195]:
def bertify(sent):
    print(sent)
    errors = find_errors(sent)
    corrections = []
    for e in errors:
        temp_sent = sent.replace(e, "<mask>")
        iceberts = icebert(temp_sent)
        print(temp_sent)
        ib_suggestions = [icb["token_str"] for icb in iceberts]
        print(ib_suggestions)
        lev_dists = []
        for s in ib_suggestions:
            d = distance(e, s, len(e), len(s))
            lev_dists.append(d)
            print(f"{e} -> {s}: {d}")
        corrections.append(ib_suggestions[lev_dists.index(min(lev_dists))])
    
    for i in range(len(errors)):
        sent = sent.replace(errors[i], corrections[i])
    return sent
    
def silent_bertify(sent):
    errors = find_errors(sent)
    corrections = []
    for e in errors:
        temp_sent = sent.replace(e, "<mask>")
        iceberts = icebert(temp_sent)
        ib_suggestions = [icb["token_str"] for icb in iceberts]
        lev_dists = []
        for s in ib_suggestions:
            d = distance(e, s, len(e), len(s))
            lev_dists.append(d)
        corrections.append(ib_suggestions[lev_dists.index(min(lev_dists))])
    
    for i in range(len(errors)):
        sent = sent.replace(errors[i], corrections[i])
    return sent


In [205]:
print(bertify("Þeta er ein stór villa"))

Þeta er ein stór villa
<mask> er ein stór villa
[' Þetta', ' Það', ' Hér', ' þetta', ' Ég']
Þeta ->  Þetta: 2
Þeta ->  Það: 4
Þeta ->  Hér: 4
Þeta ->  þetta: 3
Þeta ->  Ég: 4
 Þetta er ein stór villa


In [207]:
print(silent_bertify("ég er bara ótrúlega fljóur að skrifa"))
print(silent_bertify("Get ég skrifað setningu án villna")) # vá rómantíkus bara
print(silent_bertify("svo virðist ekki vera"))
print(silent_bertify("Hvað slær klukkn herra minn")) # ???

ég er bara ótrúlega  fljót að skrifa
Get ég skrifað  texta án  þín
svo virðist ekki vera
Hvað slær  hann herra minn


# 4. QnA

In [198]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="saattrupdan/icebert-texas-squad-is")

In [199]:
qa_context = """
Hæfi, undirbúningur og ábyrgð við sölu á Íslandsbanka
Í ljósi þess að einkahlutafélag föður fjármála- og efnahagsráðherra var á meðal kaupenda að 22,5% hlut ríkisins í Íslandsbanka sem seldur var í mars 2022 brast hann hæfi þegar hann samþykkti tillögu Bankasýslunnar um söluna. Þetta er niðurstaða umboðsmanns Alþingis í áliti hans 5. október 2023.
Að mati umboðsmanns getur það ekki haggað niðurstöðunni þótt ekkert hafi komið fram sem gefi tilefni til að draga í efa staðhæfingu ráðherra um að honum hafi á þeim tíma verið ókunnugt um þátttöku félagsins. Þá geti það heldur ekki ráðið úrslitum þótt hætta á því að ómálefnaleg sjónarmið hefðu áhrif á afstöðu ráðherra væri minni en ella sökum þess hvernig sölumeðferðinni var háttað, þáttur félagsins í heildarsölunni óverulegur og ganga verði út frá því að það hafi setið við sama borð og sambærilegir bjóðendur þegar Bankasýslan tók afstöðu til einstakra tilboða.
Umboðsmaður telur sig ekki hafa forsendur til að leggja mat á þá staðhæfingu ráðherra að óraunhæft hefði verið að skoða tengsl hans við einstaka bjóðendur með hliðsjón af því hvernig sölunni var fyrir komið. Í því sambandi bendir hann hins vegar á mikilvægi þess að undirbúningi mála sé hagað þannig að framkvæmd þeirra samrýmist lögum. Telji stjórnvöld rök standa til þess að haga beri meðferð máls á þann hátt að reglur um sérstakt hæfi eigi ekki við um tiltekin atriði beri að leita viðeigandi leiða í því sambandi. Ráðherra geti t.a.m. lagt fram frumvarp til lagabreytinga í þessu skyni og sé það þá Alþingis að meta hvort tilteknar aðstæður réttlæti að tilteknir eða allir þættir málsmeðferðar séu undanskildir reglum um sérstakt hæfi.
Umboðsmaður telur að það hefði verið í betra samræmi við upplýsingalög og vandaða stjórnsýsluhætti að afstaða til þess hvernig reglur um sérstakt hæfi horfðu við sölunni hefði komið fram í skriflegum gögnum við undirbúning málsins. Þannig hefði m.a. Alþingi gefist kostur á að gera athugasemdir við þetta atriði. Annmarkar að þessu leyti hafi skapað hættu á að ákvörðun ráðherra um að samþykkja tillögu Bankasýslunnar um söluna samrýmdist ekki reglum um sérstakt hæfi og þá með þeim afleiðingum að grafið væri undan trausti almennings á þessari ráðstöfun ríkisins. Er í því tilliti vísað til almenns tilgangs hæfisreglna og þeirra markmiða sem Alþingi hefur sérstaklega stefnt að með löggjöf á þessu sviði.
Það er álit umboðsmanns að stjórnsýsla ráðherra við undirbúning sölumeðferðarinnar hafi ekki verið í nægilega góðu samræmi við stjórnunar- og eftirlitsskyldu hans gagnvart Bankasýslu ríkisins m.t.t. þess hvernig reglur um sérstakt hæfi horfðu við. Í þessu sambandi tekur umboðsmaður fram að þáttur Bankasýslunnar, sem annaðist undirbúning sölunnar, hafi ekki verið til sjálfstæðrar skoðunar.
Umboðsmaður vísar til þess að ráðherra hafi upplýst að unnið sé að að nýju regluverki um ráðstöfun ríkisins á hlutum í fjármálafyrirtækjum og í þeirri vinnu sé m.a. til skoðunar hvernig aðkomu ráðherra verði best háttað við ráðstöfun á borð við þá sem hér um ræðir og þá að fenginni reynslu og þeim lærdómi sem dreginn verði af henni. Umboðsmaður mælist til þess að ráðherra hafi álitið í huga við þessa endurskoðun sem og frekari sölu hluta í fjármálafyrirtækjum. Þá er áréttað að með umfjöllun sinni hafi umboðsmaður ekki tekið afstöðu til hugsanlegra einkaréttarlegra afleiðinga þeirra lagalegu annmarka sem um er fjallað.
"""

def question(q):
    answer = pipe({"question": q,"context": qa_context})['answer']
    return f"{q} -> {answer}"

print(question("Hvenær fór sala á íslandsbanka fram?"))
print(question("Hvar brást Bjarna?"))
print(question("Hver átti einkahlutafélag sem var á meðal kaupenda?"))
print(question("Hvernig vill umboðsmaður að framkvæmd sé hagað?"))
print(question("Hvar býr umboðsmaður")) # plot twist

Hvenær fór sala á íslandsbanka fram? -> mars 2022
Hvar brást Bjarna? -> þegar hann samþykkti tillögu Bankasýslunnar um söluna
Hver átti einkahlutafélag sem var á meðal kaupenda? -> föður fjármála- og efnahagsráðherra
Hvernig vill umboðsmaður að framkvæmd sé hagað? -> þannig að framkvæmd þeirra samrýmist lögum
Hvar býr umboðsmaður -> einkahlutafélag föður fjármála- og efnahagsráðherra


Módelið virðist skilja hnitmiðaðar spurningar mjög vel, það hefði verið áhugavert að sjá það vinna með fleiri nefnda aðila og athuga hvort það þekkti þá í sundur.

# 5. Lokaverkefni

Ég er að pæla í að búa til gagnasafn af íslesnkum lagatextum. Þetta yrði samansafn af dægurlögum, popplögum og í raun bara öllu sem ég get komið höndum mínum yfir.

Til þess að búa til þetta gagnasafn mun ég fara í gegnum þær fjölmörgu síður á netinu sem innihalda texta, guitarparty, snerpa, glatkistan, sol.heimsnet, og skrapa af þeim textana. Ég hugsa að ég myndi reyna að nota beautiful soup fyrir þetta, hef ekki notað þann pakka áður þannig það verður áhugavert, en annars hef ég meiri reynslu í javascript þannig það væri möguleiki líka.
Ég myndi svo hreinsa gögnin og hugsanlega þjálfa generative módel á þeim.

Hér er dæmi um útfærslu, hún er ekki góð en ætti að virka með smá fiffi

In [239]:
from bs4 import BeautifulSoup
import urllib3

page = urllib3.request("GET", "https://www.snerpa.is/allt_hitt/textasafn/A_kranni/")
html_data = page.data.decode("utf-8")
soup = BeautifulSoup(html_data, 'html.parser')
def simple_cleaner (sent):
    sent = re.sub(r'[^ \w\.]', '',sent)
    if sent == "": return False
    return sent

text = [t for t in soup.article.text.split("\n") if t != ""]
text = list(filter(simple_cleaner, text))
text

['Á kránni',
 '\t"Ó, pabbi minn kæri, æ komdu með mér heim,\r',
 'sko, klukkan er senn orðin eitt.\r',
 'Þú lofaðir í morgun að koma snemma í kveld,\r',
 'á knæpunni að tefja ekki neitt.\r',
 'Nú er eldurinn dauður og allt er orðið kalt,\r',
 'og enn bíður mamma eftir þér,\r',
 'hún situr með hann Villa litla, sjúkur er hann;\r',
 'og sáralítil hjálp er að mér"-\r',
 'Viðlag:  Kom heim, kom heim,\r',
 'æ, pabbi minn kæri, kom heim.',
 '"Ó, pabbi minn kæri, æ komdu með mér heim,\r',
 'sko, klukkan slær tvö - fylgdu mér,\r',
 'það kólnar, og hann Villi litli er veikari en fyrr\r',
 'hann var þó að spyrja eftir þér - ',
 'Og segir það hún mamma mín, ef þyngi honum enn,\r',
 'að þá muni hann deyja í nótt,\r',
 'og þessar fréttir bað hún mig að bera til þín,\r',
 'æ, blessaður komdu nú fljótt.\r',
 'Viðlag...',
 '"Ó, pabbi minn kæri, æ komdu með mér heim,\r',
 'sko klukkan er senn orðin þrjú,\r',
 'og tíminn er svo langur og tómlegt heima er allt,\r',
 'við tvær erum aleinar nú.',
 'Því Vil